In [0]:
import os, warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data.', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data./t10k-images-idx3-ubyte.gz
Extracting ./mnist/data./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.images.shape

(55000, 784)

In [0]:
# 28 * 28 * 1(색 하나)로 구성된 ?만큼의 카드
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# 아웃풋은 10개(카테고리가 0부터 9까지)
Y = tf.placeholder(tf.float32, [None, 10])

## Conv -(Relu) - Pooling

In [5]:
# 3 * 3 * 1의 필터를 32개(왜 32, 3, 3, 1이 아니지?)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
# 입력값과 필터를 곱하고 원래 결과값을 나오게 함
L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding = 'SAME')
print(L1) # 패딩은 전과 같은 결과가 나오게 하고, X와 W1을 곱해줌

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)


In [6]:
L1 = tf.nn.relu(L1) # relu로 바꿔줌
print(L1)

Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)


In [7]:
# ?, 28, 28, 32의 28, 28만 2로 해서 크기를 줄여주기
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
print(L1)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


## Conv -(Relu) - Pooling

In [8]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
print(L2)

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)


In [9]:
L2 = tf.nn.relu(L2)
print(L2)

Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)


In [10]:
L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],
                   padding = 'SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


## Conv -(Relu) - Pooling

In [11]:
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
L3 = tf.nn.conv2d(L2, W3, strides = [1, 1, 1, 1], padding = 'SAME')
print(L3)

Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)


In [12]:
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)


In [13]:
L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],
                    padding = 'SAME')
print(L3)

Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)


## FC

In [14]:
# W4에 4 * 4 * 128, 256 행렬 생성 후 0.01로 초기화
W4 = tf.Variable(tf.random_normal([4 * 4 * 128, 256], stddev = 0.01))
# L2를 -1, 7 * 7 * 64로 변환
L4 = tf.reshape(L3, [-1, 4 * 4 * 128])
print(L4)

Tensor("Reshape:0", shape=(?, 2048), dtype=float32)


In [15]:
# (?, 4 * 4 * 128)(4 * 4 * 128, 256) dot product
L4 = tf.matmul(L4, W4)
print(L4)

Tensor("MatMul:0", shape=(?, 256), dtype=float32)


In [16]:
# 렐루로 변환
L4 = tf.nn.relu(L4)
print(L4)

Tensor("Relu_3:0", shape=(?, 256), dtype=float32)


In [0]:
# (?, 256)(256, 10) dot product
W5 = tf.Variable(tf.random_normal([256, 10], stddev = 0.01))
model = tf.matmul(L4, W5)

## 비용, 최적화 함수

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model,
                                                                labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [19]:
mnist.train.num_examples

55000

In [20]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
total_batch

550

In [21]:
%%time
a = 0
epoch = 31
for j in range(1, epoch):
  total_cost = 0

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    
    _, cost_val = sess.run([optimizer, cost],
                          feed_dict = {X: batch_xs,
                                      Y: batch_ys})
  
    total_cost += cost_val
  print('Epoch: {} \t Avg. cost:{:.3f}'.format(j, total_cost / total_batch))
  print('최적화 {}번 완료'.format(j))
print('최적화 끝')


Epoch: 1 	 Avg. cost:0.343
최적화 1번 완료
Epoch: 2 	 Avg. cost:0.088
최적화 2번 완료
Epoch: 3 	 Avg. cost:0.059
최적화 3번 완료
Epoch: 4 	 Avg. cost:0.043
최적화 4번 완료
Epoch: 5 	 Avg. cost:0.034
최적화 5번 완료
Epoch: 6 	 Avg. cost:0.028
최적화 6번 완료
Epoch: 7 	 Avg. cost:0.021
최적화 7번 완료
Epoch: 8 	 Avg. cost:0.018
최적화 8번 완료
Epoch: 9 	 Avg. cost:0.015
최적화 9번 완료
Epoch: 10 	 Avg. cost:0.015
최적화 10번 완료
Epoch: 11 	 Avg. cost:0.011
최적화 11번 완료
Epoch: 12 	 Avg. cost:0.011
최적화 12번 완료
Epoch: 13 	 Avg. cost:0.011
최적화 13번 완료
Epoch: 14 	 Avg. cost:0.008
최적화 14번 완료
Epoch: 15 	 Avg. cost:0.008
최적화 15번 완료
Epoch: 16 	 Avg. cost:0.007
최적화 16번 완료
Epoch: 17 	 Avg. cost:0.009
최적화 17번 완료
Epoch: 18 	 Avg. cost:0.006
최적화 18번 완료
Epoch: 19 	 Avg. cost:0.006
최적화 19번 완료
Epoch: 20 	 Avg. cost:0.006
최적화 20번 완료
Epoch: 21 	 Avg. cost:0.005
최적화 21번 완료
Epoch: 22 	 Avg. cost:0.005
최적화 22번 완료
Epoch: 23 	 Avg. cost:0.004
최적화 23번 완료
Epoch: 24 	 Avg. cost:0.006
최적화 24번 완료
Epoch: 25 	 Avg. cost:0.003
최적화 25번 완료
Epoch: 26 	 Avg. cost:0.006
최적화 26번 완료
Epoc

In [22]:
Conv, pooling 3번
필터수 변경 32, 64
에폭스 30

SyntaxError: ignored